In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.io as io

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
PATH = 'ml/results_rf_300922_rich_region_ncx_imp.csv' # change to path of interest
#PATH = 'ml/results_rf_011022_rich_region_ca1_imp.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'dev_auc']].quantile(q=[0.25, 0.5, 0.75])

auc   dev_auc
restriction modality      chunk_size                         
complete    morphological 50.0       0.25  0.979107  0.931548
                                     0.50  0.989473  1.000000
                                     0.75  0.993479  1.000000
            spatial       25.0       0.25  0.859633  0.833333
                                     0.50  0.892711  0.928571
                                     0.75  0.917919  1.000000
            temporal      1600.0     0.25  0.835152  0.785714
                                     0.50  0.856690  0.857143
                                     0.75  0.871988  0.928571

In [4]:
wf_ca1 = complete[complete.modality == 'morphological'].dev_auc.to_numpy()
wf_ncx = complete[complete.modality == 'morphological'].auc.to_numpy()

temp_ca1 = complete[complete.modality == 'temporal'].dev_auc.to_numpy()
temp_ncx = complete[complete.modality == 'temporal'].auc.to_numpy()

spat_ca1 = complete[complete.modality == 'spatial'].dev_auc.to_numpy()
spat_ncx = complete[complete.modality == 'spatial'].auc.to_numpy()

statistic, p_val = stats.mannwhitneyu(wf_ca1, wf_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(temp_ca1, temp_ncx)
print(f"Mann-Whitney statistical test results for temporal are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(spat_ca1, spat_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")
print()

Mann-Whitney statistical test results for WF are p-value=2.710606856804625e-05 (statistic=678.0)

Mann-Whitney statistical test results for temporal are p-value=0.2292378148407504 (statistic=1142.0)

Mann-Whitney statistical test results for spatial are p-value=0.02767351210046492 (statistic=972.0)



In [5]:
cs_50 = complete[complete.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = complete[complete.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = complete[complete.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

In [6]:
wf_diff = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

wf_median, wf_prec25, wf_prec75 = np.percentile(wf_diff, [50, 25, 75])
print(f"Median of WF column is {wf_median: .3f} [{wf_prec25: .3f}, {wf_prec75: .3f}]")
temp_median, temp_prec25, temp_prec75 = np.percentile(temp_diff, [50, 25, 75])
print(f"Median of temporal column is {temp_median: .3f} [{temp_prec25: .3f}, {temp_prec75: .3f}]")
spat_median, spat_prec25, spat_prec75 = np.percentile(spat_diff, [50, 25, 75])
print(f"Median of spatial column is {spat_median: .3f} [{spat_prec25: .3f}, {spat_prec75: .3f}]\n")

statistic, p_val = stats.wilcoxon(wf_dev_aucs, wf_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for wf are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs, spat_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs, temp_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for temporal p-value={p_val} (statistic={statistic})\n")

Median of WF column is  0.637 [-4.799,  1.037]
Median of temporal column is  3.767 [-9.271,  8.441]
Median of spatial column is  3.858 [-4.174,  8.011]

Wilcoxon statistical test results for wf are p-value=0.3129442963477801 (statistic=688.0)
Wilcoxon statistical test results for spatial are p-value=0.02646556476380306 (statistic=838.0)
Wilcoxon statistical test results for temporal p-value=0.3095438064717788 (statistic=689.0)



In [7]:
statistic, p_val = stats.wilcoxon(wf_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf inter-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial inter-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal inter-region auc are p-value={p_val} (statistic={statistic})")

statistic, p_val = stats.wilcoxon(wf_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf intra-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial intra-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal intra-region auc are p-value={p_val} (statistic={statistic})")

Mann-Whitney statistical test results for wf inter-region auc are p-value=7.538154154994469e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial inter-region auc are p-value=7.556929455863566e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal inter-region auc are p-value=7.556929455863566e-10 (statistic=0.0)
Mann-Whitney statistical test results for wf intra-region auc are p-value=1.0615816718159337e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial intra-region auc are p-value=6.683205687297161e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal intra-region auc are p-value=7.349358852455706e-10 (statistic=0.0)
